In [1]:
import numpy as np
import pandas as pd
import requests
import xlsxwriter
import math

## Importing Our List of Stocks


There's a static version of the S&P 500 constituents available here. [Click this link to download them now](https://drive.google.com/file/d/1ZJSpbY69DVckVZlO9cC6KkgfSufybcHN/view?usp=sharing). Move this file into the `starter-files` folder so it can be accessed by other files in that directory.

Now it's time to import these stocks to our Jupyter Notebook file.

In [2]:
stocks=pd.read_csv('sp_500_stocks.csv')

## Acquiring an API Token

Now it's time to import our IEX Cloud API token. This is the data provider that we will be using throughout this course.

API tokens (and other sensitive information) should be stored in a `secrets.py` file that doesn't get pushed to your local Git repository. We'll be using a sandbox API token in this course, which means that the data we'll use is randomly-generated and (more importantly) has no cost associated with it.

[Click here](http://nickmccullum.com/algorithmic-trading-python/secrets.py) to download your `secrets.py` file. Move the file into the same directory as this Jupyter Notebook before proceeding.

In [3]:
from secrets import IEX_CLOUD_API_TOKEN

In [7]:
symbol='AAPL'
api_url = f'https://sandbox.iexapis.com/stable/stock/{symbol}/quote?token={IEX_CLOUD_API_TOKEN}'
data=requests.get(api_url).json()
print(data)

{'symbol': 'AAPL', 'companyName': 'Apple Inc', 'primaryExchange': ')AR SGSTBO L(C/NEMNALETQLDGAA KES', 'calculationPrice': 'close', 'open': 126.35, 'openTime': 1618713693317, 'openSource': 'lfcoiiaf', 'close': 124.53, 'closeTime': 1660903367932, 'closeSource': 'lfciaoif', 'high': 128.5, 'highTime': 1631887598345, 'highSource': 'rde ud mel5iytaeep c1ni', 'low': 124.21, 'lowTime': 1690788342043, 'lowSource': 'rpc al i1e netdeyedumi5', 'latestPrice': 121.15, 'latestSource': 'Close', 'latestTime': 'March 4, 2021', 'latestUpdate': 1677951757939, 'latestVolume': 186339130, 'iexRealtimePrice': 122.74, 'iexRealtimeSize': 103, 'iexLastUpdated': 1675591758661, 'delayedPrice': 124.12, 'delayedPriceTime': 1615198273232, 'oddLotDelayedPrice': 122.78, 'oddLotDelayedPriceTime': 1693055855109, 'extendedPrice': 119.01, 'extendedChange': -1.45, 'extendedChangePercent': -0.01201, 'extendedPriceTime': 1633396623179, 'previousClose': 122.38, 'previousVolume': 114456704, 'change': -1.96, 'changePercent': -0

## Parsing Our API Call

The API call that we executed in the last code block contains all of the information required to build our equal-weight S&P 500 strategy. 

With that said, the data isn't in a proper format yet. We need to parse it first.

In [17]:
price=data['latestPrice']
market_cap=data['marketCap']

## Adding Our Stocks Data to a Pandas DataFrame

The next thing we need to do is add our stock's price and market capitalization to a pandas DataFrame. Think of a DataFrame like the Python version of a spreadsheet. It stores tabular data.

In [18]:
my_columns=['Ticker','Stock Price','Market Capitalization','Number of shares to buy']
final_dataframe=pd.DataFrame(columns=my_columns)
final_dataframe

,Ticker,Stock Price,Market Capitalization,Number of shares to buy


In [21]:
final_dataframe.append(
pd.Series([symbol,price,market_cap,'N/A'],index=my_columns),ignore_index=True
)

,Ticker,Stock Price,Market Capitalization,Number of shares to buy
0,AAPL,126.73,2143655186921,N/A


## Looping Through The Tickers in Our List of Stocks

Using the same logic that we outlined above, we can pull data for all S&P 500 stocks and store their data in the DataFrame using a `for` loop.

In [27]:
final_dataframe=pd.DataFrame(columns=my_columns)
for stock in stocks['Ticker']:
    api_url = f'https://sandbox.iexapis.com/stable/stock/{stock}/quote?token={IEX_CLOUD_API_TOKEN}'
    data=requests.get(api_url).json()
    final_dataframe=final_dataframe.append(
        pd.Series(
            [
                stock,
                data['latestPrice'],
                data['marketCap'],
                'N/A'
            ],
            index=my_columns)
        ,ignore_index=True)
    

In [30]:
final_dataframe

,Ticker,Stock Price,Market Capitalization,Number of shares to buy
0,A,123.99,36555479136,N/A
1,AAL,17.48,10674137564,N/A
2,AAP,158.89,11025885421,N/A
3,AAPL,124.94,2129163852118,N/A
4,ABBV,111.56,191824505894,N/A
...,...,...,...,...
500,YUM,108.40,33349227985,N/A
501,ZBH,142.10,29455296130,N/A
502,ZBRA,392.53,21106951665,N/A
503,ZION,42.08,7106090010,N/A


In [29]:
def chunks(lst,n):
    for i in range(0,len(lst),n):
        yield lst[i:i+n]

In [33]:
symbol_groups=list(chunks(stocks['Ticker'],100))
symbol_strings=[]
for i in range(len(symbol_groups)):
    symbol_strings.append(','.join(symbol_groups[i]))
final_dataframe=pd.DataFrame(columns=my_columns)

for symbol_string in symbol_strings:
    batch_api_call_url=f'https://sandbox.iexapis.com/stable/stock/market/batch/?types=quote&symbols={symbol_string}&token={IEX_CLOUD_API_TOKEN}'
    data=requests.get(batch_api_call_url).json()
    for symbol in symbol_string.split(','):
        final_dataframe=final_dataframe.append(
            pd.Series(
                [symbol,
                data[symbol]['quote']['latestPrice'],
                data[symbol]['quote']['marketCap'],
                'N/A'
                ],
                index=my_columns),
            ignore_index=True
        )
final_dataframe

    

,Ticker,Stock Price,Market Capitalization,Number of shares to buy
0,A,124,37211115888,N/A
1,AAL,17.89,10485046058,N/A
2,AAP,164.19,11116857328,N/A
3,AAPL,126.54,2160298424955,N/A
4,ABBV,107.91,192744012239,N/A
...,...,...,...,...
500,YUM,107.43,32447214701,N/A
501,ZBH,144.96,30296481143,N/A
502,ZBRA,382.38,20751015731,N/A
503,ZION,43.69,7168927158,N/A


## Calculating the Number of Shares to Buy

As you can see in the DataFrame above, we stil haven't calculated the number of shares of each stock to buy.

We'll do that next.

In [37]:
portfolio_size=input('enter the value of portfolio:')
try:
    val=float(portfolio_size)
except ValueError:
    print("that's not a number! Please enter a number")
    portfolio_size=input('enter the value of portfolio:')
    val=float(portfolio_size)

enter the value of portfolio:1000000


In [44]:
position_size=val/len(final_dataframe.index)
for i in range(len(final_dataframe.index)):
    final_dataframe.loc[i, 'Number of shares to buy'] = math.floor(position_size / final_dataframe['Stock Price'][i])
final_dataframe

,Ticker,Stock Price,Market Capitalization,Number of shares to buy,Number Of Shares to Buy,Number Of shares to Buy
0,A,124,37211115888,15,15.0,15.0
1,AAL,17.89,10485046058,110,110.0,110.0
2,AAP,164.19,11116857328,12,12.0,12.0
3,AAPL,126.54,2160298424955,15,15.0,15.0
4,ABBV,107.91,192744012239,18,18.0,18.0
...,...,...,...,...,...,...
500,YUM,107.43,32447214701,18,18.0,18.0
501,ZBH,144.96,30296481143,13,13.0,13.0
502,ZBRA,382.38,20751015731,5,5.0,5.0
503,ZION,43.69,7168927158,45,45.0,45.0


In [46]:
final_dataframe.drop(['Number Of Shares to Buy', 'Number Of shares to Buy'], axis = 1,inplace=True)
final_dataframe

,Ticker,Stock Price,Market Capitalization,Number of shares to buy
0,A,124,37211115888,15
1,AAL,17.89,10485046058,110
2,AAP,164.19,11116857328,12
3,AAPL,126.54,2160298424955,15
4,ABBV,107.91,192744012239,18
...,...,...,...,...
500,YUM,107.43,32447214701,18
501,ZBH,144.96,30296481143,13
502,ZBRA,382.38,20751015731,5
503,ZION,43.69,7168927158,45
